In [1]:
pip install sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 101.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21714 sha256=4e173ba0fc2c938297537d3149f2db1b37c941931e2eba7a7823a4d2b0b2df2d
  Stored in directory: /root/.cache/pip/wheels/5f/0d/0c/2062d8c1758b4b1a2e42b4a63e6660d9ec2ba9463cfee9eeab
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=760bddf4e54a4cd07a4cd004a601143ade123749bfebe92347dd4ad11f795e42
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529

In [6]:
import openpyxl
from openpyxl.utils import get_column_letter
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.utils import get_stop_words

def summarize_excel_file(file_path, cell_range, sentences_count=5, threshold=0.1, output_dir="."):
    # Load the Excel file and read the three cells
    wb = openpyxl.load_workbook(file_path)
    ws = wb.active
    cell_values = [cell.value for row in ws[cell_range] for cell in row]

    # Concatenate the cell values into a single string
    text = "\n".join(str(cell) for cell in cell_values)

    # Set up the summarizer
    LANGUAGE = "english"
    tokenizer = Tokenizer(LANGUAGE)
    parser = PlaintextParser.from_string(text, tokenizer)
    summarizer = LexRankSummarizer()

    # Set the summarizer parameters
    summarizer.stop_words = get_stop_words(LANGUAGE)
    summarizer.threshold = threshold

    # Generate the summary
    summary = ""
    for sentence in summarizer(parser.document, sentences_count):
        summary += str(sentence) + " "

    # Save the summary to a file
    output_file = f"{output_dir}/metareview.xlsx"
    try:
        wb_output = openpyxl.load_workbook(output_file)
    except FileNotFoundError:
        wb_output = openpyxl.Workbook()
    ws_output = wb_output.active
    column_letter = 'A'
    row_number = ws_output.max_row + 1
    cell = ws_output[f"{column_letter}{row_number}"]
    cell.value = summary.strip()

    # Save the output file
    wb_output.save(output_file)

    # Save the summary to a separate file
    output_path = f"{output_dir}/{file_path.split('/')[-1].replace('.xlsx', '')}_lexrank_summary_{cell_range}.txt"
    with open(output_path, "w") as f:
        f.write(summary)

    print(f"Summary saved to {output_path}")


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
import pandas as pd
import openpyxl

file_path = "/content/data2020.xlsx"
for i in range(2, 4):
    cell_range = f"C{i}:E{i}"
    summarize_excel_file(file_path, cell_range)







Summary saved to ./data2020_lexrank_summary_C2:E2.txt
Summary saved to ./data2020_lexrank_summary_C3:E3.txt
